In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using JLD2

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(dirname(parent_dir))
data_dir = string(grand_pa_dir, "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end
println("data_dir:",data_dir)

data_dir:/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/


## Functions

In [3]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [ ]:
project_name = "Pricing_Promotion/"
seed = 1;
N = 3; # num of products
N_u = 1; 
K = 10;
S_train = 500;
S_test = 1000;
P_bar = 30.0;
iterations = 10;
# RO_coef_all = [0.0];
RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.15,0.2,0.25,0.3];
Random.seed!(seed)

In [20]:
P_dag = round.(rand(N, K) .* P_bar; digits=2)

3×10 Matrix{Float64}:
 2.66  1.35  4.79  1.75  3.85  4.54  4.36  0.4   0.22  4.95
 4.34  4.62  2.65  4.09  0.32  0.41  4.19  4.38  4.78  3.99
 0.95  0.55  3.52  3.64  3.51  2.47  4.41  2.79  1.14  1.31

### offdiag is mix

In [5]:
offdiag_sign = "mix";
max_offdiag = 1.0;
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed);

In [6]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_name,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);

- Generate Data

In [7]:
function Generate_Params(N,N_u)
    alpha = [1,1,1]
    beta = [0.1,0.2,0.3]
    gamma = [0.8,0.3,0.5]
    return alpha, beta,gamma
end

Generate_Params (generic function with 1 method)

In [8]:
alpha, beta,gamma = Generate_Params(N,N_u)

([1, 1, 1], [0.1, 0.2, 0.3], [0.8, 0.3, 0.5])

- Run Oracle

In [38]:
model = Model(Mosek.Optimizer)
set_attribute(model, "QUIET", true)
# model = Model(COPT.ConeOptimizer)
# 定义变量
@variable(model, rho_0 >= 0);                       # ρ₀ ≥ 0
@variable(model, rho[1:N] >= 0);                    # ρ_n ≥ 0
@variable(model, rho_P0[1:N]);                      # φ_n
@variable(model, rho_X0[1:N]);                       # ς = ρ₀ * u

@variable(model, rho_P[1:N]);                      # φ_n
@variable(model, rho_X[1:N]);                       # ς = ρ₀ * u

@variable(model, Y[1:N, 1:K]);                      # y_{nk}
@variable(model, Z[1:N, 1:K]);                      # z_{nk}
@variable(model, X_P[1:N, 1:K], Bin);               # x_{nk} ∈ {0,1}
@variable(model, X_PM[1:N], Bin);                   # x_{nk} ∈ {0,1}
@variable(model, W[1:N,1:N,1:K]);                   # z_{nk}


In [39]:
@constraint(model, rho_0 + sum(rho) == 1)

rho_0 + rho[1] + rho[2] + rho[3] = 1

In [40]:
for n in 1:N
    @constraint(model, [alpha[n] * rho_0 - beta[n] * rho_P0[n] + gamma[n] * rho_X0[n],rho_0,rho[n]] in MOI.ExponentialCone())
end

In [41]:
@constraint(model, rho_P0 .== sum(X_P .* P_dag, dims=2))

3×1 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 rho_P0[1] - 2.66 X_P[1,1] - 1.35 X_P[1,2] - 4.79 X_P[1,3] - 1.75 X_P[1,4] - 3.85 X_P[1,5] - 4.54 X_P[1,6] - 4.36 X_P[1,7] - 0.4 X_P[1,8] - 0.22 X_P[1,9] - 4.95 X_P[1,10] = 0
 rho_P0[2] - 4.34 X_P[2,1] - 4.62 X_P[2,2] - 2.65 X_P[2,3] - 4.09 X_P[2,4] - 0.32 X_P[2,5] - 0.41 X_P[2,6] - 4.19 X_P[2,7] - 4.38 X_P[2,8] - 4.78 X_P[2,9] - 3.99 X_P[2,10] = 0
 rho_P0[3] - 0.95 X_P[3,1] - 0.55 X_P[3,2] - 3.52 X_P[3,3] - 3.64 X_P[3,4] - 3.51 X_P[3,5] - 2.47 X_P[3,6] - 4.41 X_P[3,7] - 2.79 X_P[3,8] - 1.14 X_P[3,9] - 1.31 X_P[3,10] = 0

In [42]:
for n in 1:N
    for k in 1:K
        @constraint(model, 0 <= rho_P0[n])
        @constraint(model, rho_P0[n] <= X_P[n, k])
        @constraint(model, rho_P0[n] >= rho_0 - (1 - X_P[n, k]))
        @constraint(model, rho_P0[n] <= rho_0)
    end
end

In [43]:
@constraint(model, X_P * ones(K) .== 1)

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 X_P[1,1] + X_P[1,2] + X_P[1,3] + X_P[1,4] + X_P[1,5] + X_P[1,6] + X_P[1,7] + X_P[1,8] + X_P[1,9] + X_P[1,10] = 1
 X_P[2,1] + X_P[2,2] + X_P[2,3] + X_P[2,4] + X_P[2,5] + X_P[2,6] + X_P[2,7] + X_P[2,8] + X_P[2,9] + X_P[2,10] = 1
 X_P[3,1] + X_P[3,2] + X_P[3,3] + X_P[3,4] + X_P[3,5] + X_P[3,6] + X_P[3,7] + X_P[3,8] + X_P[3,9] + X_P[3,10] = 1

In [44]:
for n in 1:N
    @constraint(model, 0 <= rho_X0[n])
    @constraint(model, rho_X0[n] <= X_PM[n])
    @constraint(model, rho_X0[n] >= rho_0 - (1 - X_PM[n]))
    @constraint(model, rho_X0[n] <= rho_0)
end

In [45]:
@constraint(model, sum(X_PM) == 1)

X_PM[1] + X_PM[2] + X_PM[3] = 1

In [46]:
for n in 1:N
    @constraint(model, [-alpha[n] * rho[n] + beta[n] * rho_P[n] - gamma[n] * rho_X[n], rho[n], rho_0] in MOI.ExponentialCone())
end

In [ ]:
@constraint(model, rho_P .== sum(X_P .* P_dag, dims=2))

In [ ]:

@constraint(model, v_pi0 .== Y)
for n in 1:N
        @constraint(model, 0 <= Y[n])
        @constraint(model, Y[n] <= X_PM[n])
        @constraint(model, Y[n] >= rho_0 - (1 - X_PM[n]))
        @constraint(model, Y[n] <= rho_0)
end
@constraint(model, v_phi0 .== sum(Z .* p_dag, dims=2))
for n in 1:N
    for k in 1:K
        # z_{nk} bounds
        @constraint(model, 0 <= Z[n, k])
        @constraint(model, Z[n, k] <= X_P[n, k])
        @constraint(model, Z[n, k] >= rho_0 - (1 - X_P[n, k]))
        @constraint(model, Z[n, k] <= rho_0)
    end
end

for n in 1:N
    @constraint(model, [-alpha[n] * v_sigma_i[n] + beta[n] * v_phi_i[n] - gamma[n] * v_pi_i[n], rho[n], rho_0] in MOI.ExponentialCone())
end

@constraint(model, v_sigma_i .== rho_0 .* u)
@constraint(model, v_pi_i .== Y_i)
for n in 1:N
        @constraint(model, 0 <= Y_i[n])
        @constraint(model, Y_i[n] <= X_PM[n])
        @constraint(model, Y_i[n] >= rho[n] - (1 - X_PM[n]))
        @constraint(model, Y_i[n] <= rho[n])
end
@constraint(model, v_phi_i .== sum(Z_i .* p_dag, dims=2))
for n in 1:N
    for k in 1:K
        # z_{nk} bounds
        @constraint(model, 0 <= Z_i[n, k])
        @constraint(model, Z_i[n, k] <= X_P[n, k])
        @constraint(model, Z_i[n, k] >= rho[n] - (1 - X_P[n, k]))
        @constraint(model, Z_i[n, k] <= rho[n])
    end
end


@constraint(model, sum(X_PM) == 1)

@objective(model, Max,sum(Y .* p_dag))

optimize!(model)
status = JuMP.termination_status(model)
# solution_summary(model)
if status == MOI.OPTIMAL
    obj_val = objective_value(model)
    X_val = value.(X)
    solve_time = JuMP.solve_time(model)
else
    obj_val = NaN
    X_val = ones(N,N) .* NaN
    solve_time = NaN
end

In [ ]:
function ETO_Pricing_Promotion(N,N_u,K,A,B,u,p_dag)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # model = Model(COPT.ConeOptimizer)
    # 定义变量
    @variable(model, rho_0 >= 0)                       # ρ₀ ≥ 0
    @variable(model, rho[1:N] >= 0)                    # ρ_n ≥ 0
    @variable(model, v_sigma[1:N_u])                   # ς = ρ₀ * u
    @variable(model, v_phi[1:N])                       # φ_n
    @variable(model, v_theta[1:N,1:N_u])               # ς = ρ₀ * u
    @variable(model, v_pi[1:N,1:N])  
    @variable(model, Y[1:N, 1:K])                      # y_{nk}
    @variable(model, Z[1:N, 1:K])                      # z_{nk}
    @variable(model, X[1:N, 1:K], Bin)                 # x_{nk} ∈ {0,1}
    @variable(model, W[1:N,1:N,1:K])                   # z_{nk}
    
    @constraint(model, rho_0 + sum(rho) == 1)

    for n in 1:N
        @constraint(model, [A[n,:]' * v_sigma + B[n,:]' * v_phi,rho_0,rho[n]] in MOI.ExponentialCone())
    end

    @constraint(model, v_sigma .== rho_0 .* u)
    @constraint(model, v_phi .== sum(Z .* p_dag, dims=2))
    for n in 1:N
        for k in 1:K
            # z_{nk} bounds
            @constraint(model, 0 <= Z[n, k])
            @constraint(model, Z[n, k] <= X[n, k])
            @constraint(model, Z[n, k] >= rho_0 - (1 - X[n, k]))
            @constraint(model, Z[n, k] <= rho_0)
        end
    end

    for n in 1:N
        @constraint(model, [-A[n,:]' * v_theta[n,:] - B[n,:]' * v_pi[n,:], rho[n], rho_0] in MOI.ExponentialCone())
    end
    for n in 1:N
        @constraint(model, v_theta[n,:] .== rho[n] .* u)
    end
    for n in 1:N
        @constraint(model, v_pi[n,:] .== sum(W[n,:,:] .* p_dag, dims=2))
    end
    for n in 1:N
        for j in 1:N
            for k in 1:K
                # z_{nk} bounds
                @constraint(model, 0 <= W[n,j, k])
                @constraint(model, W[n,j,k] <= X[j, k])
                @constraint(model, W[n,j,k] >= rho[n] - (1 - X[j, k]))
                @constraint(model, W[n,j, k] <= rho[n])
            end
        end
    end

    # objective 
    for n in 1:N
        for k in 1:K
            # y_{nk} bounds
            @constraint(model, 0 <= Y[n, k])
            @constraint(model, Y[n, k] <= X[n, k])
            @constraint(model, Y[n, k] >= rho[n] - (1 - X[n, k]))
            @constraint(model, Y[n, k] <= rho[n])
        end
    end

    @constraint(model, sum(X,dims=2) .== 1)

    @objective(model, Max,sum(Y .* p_dag))

    optimize!(model)
    status = JuMP.termination_status(model)
    # solution_summary(model)
    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        X_val = value.(X)
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        X_val = ones(N,N) .* NaN
        solve_time = NaN
    end
    return obj_val, X_val,solve_time
end


In [ ]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

- Run ETO

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

- Run RO under different uncertainty set

In [ ]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_RO = Dict();
for RO_coef in RO_coef_all
    RST_RO_this = RST_RO["RO_coef=$(RO_coef)"]
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations];
end

println("Mean Rev_Oracle: ", mean(Rev_Oracle));
println("Mean Rev_ETO: ", mean(Rev_ETO));
Rev_RO_Avg = [mean(Rev_RO["RO_coef=$(RO_coef)"]) for RO_coef in RO_coef_all];
println("Mean Rev_RO: ", Rev_RO_Avg);

In [ ]:
ratio = Rev_RO_Avg ./ mean(Rev_ETO)
plot(RO_coef_all, ratio, marker=:o, xlabel="RO_coef", ylabel="Rev_RO_Avg / Rev_ETO_Avg", legend=false)
hline!([1.0], linestyle=:dash, color=:red, label="y=1.0")
# savefig(plt, "Rev_RO_Avg_vs_Rev_ETO_Avg.png")  # 保存为当前目录下的PNG图片

In [ ]:
plot_boxplot(RO_coef_all[1:5],Rev_ETO,Rev_RO)

### offdiag is positive

In [ ]:
# offdiag_sign = "positive";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# Random.seed!(seed)
# Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
# Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
# save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
# for RO_coef in RO_coef_all
#     Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
#     println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
# end

### offdiag is negative

In [ ]:
# offdiag_sign = "negative";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);